In [ ]:
"""
Model
"""
import os
import csv
import math
import numpy as np
import cv2
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Activation, Dropout, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LABELS_FILENAME = 'data/driving_log.csv'

"""
Functions for training
"""
# generate training data in an infinite loop - called from keras fit_generator
def genTrainingData(data):
    while True:
        src = genNormalizedData(genAugmentedViews(data))
        for sample in src:
            yield sample

# enumerate rows of csv data - returns raw row contents
def enumDrivingLog(driving_log_fname):
    with open(driving_log_fname, 'r') as labels_file:
        rdr=csv.DictReader(labels_file)
        for row in rdr:
            yield row

# filter out some rows (based on keepPercentage) with low steering values (based on threshold)
def filterDrivingLog(src, threshold, keepPercentage):
    for row in src:
        steering = float(row['steering'])
        if abs(steering) > threshold or np.random.randint(0,100) <= keepPercentage:
            yield row

# generate images that supplement the data set
def genAugmentedViews(src):
    for row in src:
        steering = float(row['steering'])
        center = readImage(row['center'])
        left = readImage(row['left'])
        right = readImage(row['right'])
        
        # center camera image
        yield (center, steering)
        
        # adjust steering for left and right cameras
        left_steer = steering + 0.25
        right_steer = steering - 0.25
        yield (left, left_steer)
        yield (right, right_steer)

        # add random shadows to center, right and left cameras
        yield (addShadow(center), steering)
        yield (addShadow(left), left_steer)
        yield (addShadow(right), right_steer)

        # flip center camera image
        yield (cv2.flip(center, 1), steering * -1.)

        # vary image brightness on each camera - to half and third of the original brightness
        yield (adjustBrightness(center, 2), steering)
        yield (adjustBrightness(center, 3), steering)
        yield (adjustBrightness(left, 2), left_steer)
        yield (adjustBrightness(left, 3), left_steer)
        yield (adjustBrightness(right, 2), right_steer)
        yield (adjustBrightness(right, 3), right_steer)

# adds a random shadow to a given image
def addShadow(img):
    y,u,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb))
    y = y.astype(np.int32)
    # create mask image
    mask = np.zeros(y.shape, dtype=np.int32)
    # compute line in slope, intercept form
    x1 = np.random.uniform() * y.shape[1]
    x2 = np.random.uniform() * y.shape[1]
    slope = float(y.shape[0]) / (x2 - x1)
    intercept = -(slope * x1)
    # assign pixels of mask
    for j in range(mask.shape[0]):
        for i in range(mask.shape[1]):
            if j > (i*slope)+intercept:
                mask[j,i] -= 80
    # apply mask
    y += mask
    y = np.clip(y, 0,255).astype(np.uint8)
    return cv2.cvtColor(cv2.merge((y,u,v)), cv2.COLOR_YCrCb2RGB)

# adjust brightness of given image (img) by a divisor (d - small int like 2,3,4)
def adjustBrightness(img, d):
    h,s,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2HSV))
    return cv2.cvtColor(cv2.merge((h,s,v // d)), cv2.COLOR_HSV2RGB)

# normalize image and steering data
# round out steering values to fall in 0.01 buckets (so problem is simplified, could even become classification problem)
# keras requires np.array for y values: convert steering to 1D np.array
def genNormalizedData(src):
    for img,steering in src:
        # round and convert from -1 : +1 to -0.5 : + 0.5
        #normSteering = round(steering,2) / 2.
        normSteering = round(steering,2)
        x = normalizeImage(img)
        y = np.array(normSteering, ndmin=1, dtype=np.float)
        yield x,y
        
"""
Operations done for every image - both training and predictions
"""
# retrive image data from disk
# returns image in RGB format
def readImage(img_fname):
    img_path = os.path.join('data',img_fname.strip())
    img = cv2.imread(img_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# convert RGB to YUV
def convColorSpace2YUV(img):
    y,u,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb))
    return cv2.merge((y,u,v))

# convert RGB to gray (keep 3rd dim as 1)
def convColorSpace2Gray(img):
    y,u,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb))
    return y.reshape(*y.shape,1)

# crop 5th of top and bottom of given image
def cropImage(img):
    ht = img.shape[0]
    cropHt = ht // 5
    return img[cropHt:ht-cropHt, :]

# normalize image data
# convert to y,u,v
# crop to remove top 5th and bottom 5th of image
# adjust pixel values to fall within -0.5 to +0.5
# keras requires 4d-tensor input: add dimension 1, h,w,d for image data
def normalizeImage(img):
    croppedImg = cropImage(img)
    yuvImg = convColorSpace2Gray(croppedImg)
    nImg = ((yuvImg / 255.0) - 0.5).astype(np.float32)
    x = nImg.reshape(1,*nImg.shape)
    return x

# de-normalize image data to RGB format for display
def imgForDisplay(img):
    img = ((img + 0.5) * 255).astype(np.uint8)
    img = img.reshape(img.shape[1:4])
    if img.shape[2] == 3: #yuv
        img = cv2.cvtColor(img, cv2.COLOR_YCrCb2RGB)
    elif img.shape[2] == 1: #grayscale
        img = img.reshape(img.shape[0:2])
    return img

data = list(filterDrivingLog(enumDrivingLog(LABELS_FILENAME), 0.01, 25))

# split into train and validation
#train, test = train_test_split(data, test_size=0.25)
#train = shuffle(train)
train = data

imageGenerator = genNormalizedData(genAugmentedViews([train[0]]))
imgListExample = list(imageGenerator)
numImagesPerSample = len(imgListExample)
numRows = len(train) * numImagesPerSample
exampleImg = imgListExample[0][0]
image_shape = exampleImg.shape
print("NumSamples: {0}, Shape:{1}".format(numRows, image_shape))

for normImg,normSteering in imgListExample:
    plt.figure()
    plt.axis('off')
    img = imgForDisplay(normImg)
    ht = img.shape[0]+10
    steering = normSteering # * 2.
    if len(img.shape) == 3: # rgb
        plt.imshow(img)
    else: # grayscale
        plt.imshow(img, cmap='gray')
    plt.text(0,ht,'Steering: %.3f' % steering)


In [ ]:
MODEL_FILE = 'data/model.h5'
if os.path.exists(MODEL_FILE):
    print("Loading from file")
    model = load_model(MODEL_FILE)
else:
    print("Creating model")
    model = Sequential()

    model.add(Conv2D(8, 5, 5, input_shape=image_shape[1:4]))
    model.add(MaxPooling2D((2,2)))
    #model.add((Dropout(0.5)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(16, 5, 5))
    model.add(MaxPooling2D((2,2)))
    #model.add((Dropout(0.5)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, 5, 5))
    model.add(MaxPooling2D((2,2)))
    #model.add((Dropout(0.5)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, 3, 3))
    model.add(MaxPooling2D((2,2)))
    #model.add((Dropout(0.5)))
    model.add(Activation('relu'))


    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='softmax'))

model.summary()


In [ ]:
# TODO: Compile and train the model here.
from keras.optimizers import SGD
from keras.optimizers import Adam
# optimizer = SGD(lr=0.0001)
optimizer = Adam()
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
history = model.fit_generator(genTrainingData(train), samples_per_epoch=numRows, nb_epoch=1, verbose=1, max_q_size=50)

In [ ]:
model.save(MODEL_FILE)